## Part 1: Book Recommendation

In [33]:
import pandas as pd

df = pd.read_csv('../data/books_data.csv')

In [35]:
len(df)

212404

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
df["Clean Title"] = df['Title'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
df['combined_features'] = df['Clean Title'].fillna('') + ' ' + df['authors'].fillna('') + ' ' + df['description'].fillna('')
# df['combined_features'] = df['Clean Title'].fillna('') + ' ' + df['description'].fillna('')
# df['combined_features'] = df['description'].fillna('') # The problem here is that some books don't have a description
df['combined_features'] = df['combined_features'].str.lower().str.replace(r'[^\w\s]', '', regex=True)

# Remove any non-unique rows
df = df.drop_duplicates(subset=['Clean Title'])

# Define variables
ID_COLUMN = 'Title'
COMPARISON_COLUMN = 'combined_features'
SPECIFIC_ID = 'Its Only Art If Its Well Hung!'

# Vectorize the comments
v = TfidfVectorizer(stop_words='english')
X = v.fit_transform(df[COMPARISON_COLUMN])

# Map IDs to indices
Id2idx = pd.Series(df.index, index=df[ID_COLUMN])

def get_most_similar(id):
    idx = Id2idx[id]
    scores = cosine_similarity(X, X[idx]).flatten()
    recommended = (-scores).argsort()[1:6]
    return df[ID_COLUMN].iloc[recommended], scores[recommended]

# Get similar items
similar = get_most_similar(SPECIFIC_ID)

# Create DataFrame with results
df_similar = pd.DataFrame({
    'ID': similar[0],
    'Score': similar[1],
    'Comment': similar[0].apply(lambda x: df[df[ID_COLUMN] == x][COMPARISON_COLUMN].values[0])
})

print('Original:')
print(df[df[ID_COLUMN] == SPECIFIC_ID][COMPARISON_COLUMN].values[0])

print(df_similar[["ID", "Score"]].head(5))

Original:
its only art if its well hung julie strain 
                                         ID     Score
209127                   Hung by the tongue  0.365535
203498                  Julie Of The Wolves  0.321138
65990                            The Return  0.290065
40737                The RETURN: THE RETURN  0.288097
167024  Gardening Without Stress and Strain  0.282020


### Evaluate the model

For now, let's keep model evaluation simple by sampling a couple recommendations and manually reviewing. Later, we'll take a more complex approach

In [88]:
# Manual review

# Sample five books to assess recommendations
sample_books = df.sample(5, random_state=20)

top_recs = []
for _, row in sample_books.iterrows():
    book_recs = get_most_similar(row["Title"])
    top_recs.append(book_recs[0])

# Print the sampled titles with top recommendations for that title
for i, rec in enumerate(top_recs):
    print(f"Book: {sample_books.iloc[i]['Title']}")
    print(f"Top rec: {rec.iloc[0]}")

Book: The Sierra Club: Mountain Light Postcard Collection: A Portfolio
Top rec: The Encyclopedia of Ancient Civilizations
Book: Starting and Succeeding in Real Estate
Top rec: The Official XMLSPY Handbook
Book: The Eye of the Abyss (Franz Schmidt, 1)
Top rec: The Art of Translating Prose
Book: The Kabbalah Pillars: A Romance of The Ages
Top rec: How To Make The Devil Obey You!!!
Book: Iridescent Soul
Top rec: Wallace Stevens: A Poet's Growth


Our recommendations seem a bit all over the place right now. In part 3, we'll do some work to improve the quality of our recommendations.

## Part 2: Composite Book Ratings

In [37]:
import pandas as pd

ratings = pd.read_csv('../data/Books_rating.csv')
ratings["Clean Title"] = ratings["Title"].str.lower().str.replace(r'[^\w\s]', '', regex=True)

In [16]:
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/adene/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy.sparse import hstack
import numpy as np

ratings_sample = ratings.sample(frac = 0.05, random_state = 42)

# TODO: Build review/text and review/summary -> review/score model
v = TfidfVectorizer(stop_words='english')
combined = ratings_sample["review/summary"].fillna("") + " " + ratings_sample["review/text"].fillna("")


# Add Vader sentiment analysis
sid_obj = SentimentIntensityAnalyzer()
ratings_sample["Polarity"] = ratings_sample["review/summary"].fillna("").apply(lambda x: sid_obj.polarity_scores(x)["compound"])

# Convert polarity series to a 2D numpy array (shape: n_samples x 1)
X_polarity = np.array(ratings_sample["Polarity"]).reshape(-1, 1)

# Get the TF-IDF features (sparse matrix)
X_text = v.fit_transform(combined)

# Combine the two using hstack so that each row is a concatenation of TF-IDF features and the polarity value.
X = hstack([X_text, X_polarity])
y = ratings_sample["review/score"]

In [39]:
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model = tree.DecisionTreeRegressor(max_depth=15)
model.fit(X_train, y_train)

test_predictions = model.predict(X_test)
train_predictions = model.predict(X_train)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
DECIMALS = 3

# Testing metrics
test_MAE = mean_absolute_error(y_test, test_predictions)
test_MSE = mean_squared_error(y_test, test_predictions)
test_RMSE = np.sqrt(test_MSE)
test_r2 = r2_score(y_test, test_predictions)
print('Testing: MAE=', round(test_MAE, DECIMALS), ', MSE=', round(test_MSE, DECIMALS), ', RMSE=', round(test_RMSE, DECIMALS), ', R-squared: ', round(test_r2, DECIMALS))

# Training metrics
train_MAE = mean_absolute_error(y_train, train_predictions)
train_MSE = mean_squared_error(y_train, train_predictions)
train_RMSE = np.sqrt(train_MSE)
train_r2 = r2_score(y_train, train_predictions)
print('Training: MAE=', round(train_MAE, DECIMALS), ', MSE=', round(train_MSE, DECIMALS), ', RMSE=', round(train_RMSE, DECIMALS), ', R-squared: ', round(train_r2, DECIMALS))

Testing: MAE= 0.766 , MSE= 1.099 , RMSE= 1.049 , R-squared:  0.236
Training: MAE= 0.708 , MSE= 0.929 , RMSE= 0.964 , R-squared:  0.358


In [ ]:
from tqdm import tqdm
# TODO: Multiple reviews -> average review/score model

# To predict the average review, let's predict individual reviews and average

# Group by book from ratings, sample 5% of total books, 
unique_titles = ratings["Clean Title"].drop_duplicates()

sample_books = unique_titles.sample(frac = 0.05, random_state = 42)

grouped = ratings.groupby("Clean Title")

abs_error = []

for title in tqdm(sample_books):
    # Retrieve all rows (i.e., reviews) for this book.
    book_reviews = grouped.get_group(title).copy()

    book_reviews.loc[:, "Polarity"] = book_reviews["review/summary"].fillna("").apply(lambda x: sid_obj.polarity_scores(x)["compound"])

    # Convert polarity series to a 2D numpy array (shape: n_samples x 1)
    X_polarity = np.array(book_reviews["Polarity"]).reshape(-1, 1)

    # Get the TF-IDF features (sparse matrix)
    X_text = v.transform(book_reviews["review/summary"].fillna("") + " " + book_reviews["review/text"].fillna(""))

    # Combine the two using hstack so that each row is a concatenation of TF-IDF features and the polarity value.
    X = hstack([X_text, X_polarity])
    y = ratings_sample["review/score"]
    
    predicted_scores = model.predict(X)
    avg_pred = np.mean(predicted_scores)
    
    # Here, we compute the actual average review score:
    avg_score = book_reviews["review/score"].mean()
    
    abs_error.append(abs(avg_pred - avg_score))

print("Mean absolute error:", round(np.mean(abs_error), 3))

## Part 3: Comprehensive Recommender System

In [ ]:
# Pairwise recommendation tuning